In [5]:
import pandas as pd 
import pickle
import string
from pymongo import MongoClient 
from datetime import datetime,timedelta
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import logging
import pymagnitude
import sys

sys.path.insert(0, "//home//nlpsomnath//NegNews//zackc//Misc Notebooks//NN_VM//lib//python3.7//site-packages//negative_news//negative_news//util//")
from fines import get_data_from_article_title, transform_article_title, load_entity_dict, generate_report_string, max_marketshare
from desm import desm_pandas_wrapper

logger = logging.getLogger(__name__)
sent_tokenizer = nltk.tokenize.sent_tokenize
word_tokenizer = nltk.tokenize.word_tokenize

stopwords = stopwords.words('english')
newStopWords = ['.','?','%','&','Google','Wells Fargo','Donald Trump','Charles Schwab','Morgan Stanley'
                ,'Credit Suisse','Reuters','Bank of America','Guggenheim','Deutsch Bank','Goldman Sachs'
                ,'Facebook','Fifth Third Bank','New York','Washington','Monday','Tuesday','Wednesday'
                ,'Thursday','Friday','Saturday','Sunday','January','February','March','April','May'
                ,'June','July','August','September','October','November','December']
stopwords.extend(newStopWords)
stemmer = SnowballStemmer("english")
punctuation = string.punctuation + "...—’“”"

#CONSTANTS
RESOURCES_PATH="/home/nlpsomnath/.local/lib/python3.5/site-packages/negative_news/resources/"
OUTPUT_PATH = "/home/nlpsomnath/NegNews/zackc/ContainerOutput/"

#Data
#classified_df =pd.read_pickle("{}".format(OUTPUT_PATH)+"DF_With_DESM_Scores_AND_CLASSIFICATIONS.pickle")
classified_df =pd.read_pickle("{}".format(OUTPUT_PATH)+"NEW_SENTLEVEL_CLASSIFICATIONS_DF.pickle")


In [6]:
classified_df.columns

Index(['penalty', 'restitution', 'sanction', 'fined', 'escalation',
       'violation', 'deficiencies', 'collude', 'suspension', 'infringement',
       'revoke', 'jobname', 'date', 'entity', 'scraper', 'source', 'title',
       'url', 'max_desm_score', 'doc_sentiment', 'entity_count', 'sentence',
       'sentence_sentiment', 'sent_index', 'sent_len', 'sent_first20',
       'classifications'],
      dtype='object')

In [2]:
#CONSTANTS
RESOURCES_PATH="/home/nlpsomnath/.local/lib/python3.5/site-packages/negative_news/resources/"
OUTPUT_PATH = "/home/nlpsomnath/NegNews/zackc/ContainerOutput/"

classified_df =pd.read_pickle("{}".format(OUTPUT_PATH)+" DF_With_DESM_Scores_AND_CLASSIFICATIONS.pickle")

#Date threshold
MIN_YEAR = 2000 
MIN_MONTH = 1
MIN_DAY = 1

avg_top_n_scores = 3
regulatory_url_list = [".gov", "finra.org"]
norm_date = datetime.now().date() - timedelta(10 * 365)

TERM_SIMILARITY_REMOVAL_THRESHOLD = 0.55
URL_FUZZY_THRESHOLD = 90
ENTITY_FUZZ_SCORE_THRESHOLD = .95
TITLE_FUZZY_THRESHOLD = 90

MODEL_WEIGHTS_DICT = {
    "WEIGHT_TERM_SIMILARITY": 0.8,
    "WEIGHT_ENTITY_COUNT": 0.1,
    "WEIGHT_SENTIMENT": 0.01,
    "WEIGHT_FINE_AMOUNT": 0.08,
    "WEIGHT_DOCUMENT_AGE": 0.02,
}

ASYMPTOTIC_OFFSETS_DICT = {
    "ASYMPTOTIC_OFFSET_ENTITY": 2 / 0.3,
    "ASYMPTOTIC_OFFSET_URL_COUNT": 2 / 0.4,
    "ASYMPTOTIC_OFFSET_TERM_SIMILARITY": 200,
}

RISK_SCORE_NORMS_DICT = {"RISK_SCORE_NORM_MIN": 0.15, "RISK_SCORE_NORM_MAX": 0.396}

THRESHOLD_RANGE_DICT = {
    "max_desm_score": {"range": [0.272, 0.385]},
    "doc_sentiment": {"range": [0.33, 0.667]},
    "age_norm": {"range": [0.413, 0.706]},
    "entity_count_norm": {"range": [0.444, 0.722]},
    "fine_amt_norm": {"range": [0.005, 0.667]},
}

ARTICLE_CLASSES_DICT = {
    "other": {
            "code": 0,
            "stems": [],
            "threshold": 0.01,
            "display_name": "Informative (No Action)", 
            "static_phrases":[],
    },
#     "complaint": {
#                 "code": 1,
#                 "stems": ["complaint", "unsatisfied", "unhappy","omitting"],
#                 "threshold": 0.4,
#                 "display_name": "Complaint (No Action)",
#                 "static_phrases": [],
#     },
#     "legal_action": {
#                 "code": 2,
#                 "stems": ["legal", "enforcement", "federal","violate"],
#                 "threshold": 0.6,
#                 "display_name": "Legal Action (Potential Action)",
#                 "static_phrases": ["enforcement action","legal action","sanction"],
#     },
    "monetary_fine_minor": {
                        "code": 3,
                        "stems": ["fine", "penalty"],
                        "threshold": 0.6,
                        "display_name": "Monetary Fine (No Action)",
                        "static_phrases": [],
    },
#     "regulatory_reference": {
#                         "code": 2,
#                         "stems": ["regulate", "oversight","Fed","comptroller"],
#                         "threshold": 0.5,
#                         "display_name": "Regulatory Reference (Potential Action)",
#                         "static_phrases": ["Financial Industry Regulatory Authority","Securities and Exchange Commission","FINRA","OCC","CFTC","FDIC","SEC","Attorney General","regulators","Federal Reserve"],
#     },
#     "m_and_a": {
#             "code": 3,
#             "stems": ["merger", "aquisition","buyout","takeover"],
#             "threshold": 0.9,
#             "display_name": "Merger/Acquisition (No Action)",
#             "static_phrases": ["merger","aquired"],
#     },
    "monetary_fine_major": {"code": 7,
                            "stems": [],
                            "threshold": 0.6,
                            "display_name": "Monetary Fine (Action Required)",
                            "static_phrases":[],
    },
#     "mgmt_change": {
#                 "code": 7,
#                 "stems": ["resigned","restructuring"],
#                 "threshold": 0.8,
#                 "display_name": "Management Change (Potential Action)",
#                 "static_phrases":["management shakeup","reshuffles management"
#                                   ,"executive shakeup","leadership change","restructuring","resigned"
#                                   ,"replace some of its board of directors","next president and chief executive"],
#     },
    "statutory_disqual": {
                        "code": 9,
                        "stems": ["violation","disqualify" ,"statutory","barred","suspend","suspension"],
                        "threshold": 0.9,
                        "display_name": "Statutory Disqualification (Action Required)",
                        "static_phrases":["willfully violated","failing to disclose","omitting to disclose","consent order",
                                            "registration as a broker-dealer was withdrawn","Statutory Disqualification",
                                            "cease-and-desist","Securities Act of","violation of federal securities laws",
                                            "Securities Exchange Act of 1934","put restrictions on its business",
                                            "entering into a consent order","breached fiduciary duties","suspended from",
                                            "trading ban","registration ban","Trading Ban","identified the deficiencies"],
    },
#     "trade_suspension": {
#                         "code": 9,
#                         "stems": ["suspend", "suspension"],
#                         "threshold": 0.95,
#                         "static_phrases": ["trade suspension","suspended trading","15c2-11"],
#                         "display_name": "Trade Suspension (Action Required)",
#     },
}

entity_dict = load_entity_dict("//home//nlpsomnath//NegNews//zackc//entity_dict.pickle")

In [3]:
##FUNCTIONS 
import dateparser

def date_extract_pandas_wrapper(row):
    # logger.debug('Extracting date: initial value is "{}" with type "{}"'.format(row['date'], type(row['date'])))

    if isinstance(row["date"], str):
        extracted_date = dateparser.parse(row["date"])
        if extracted_date is not None:
            # logger.debug('Dateparser extracted the following date: {}'.format(extracted_date))
            return datetime(extracted_date.year, extracted_date.month, extracted_date.day)
    elif isinstance(row["date"], datetime):
        return row["date"]
    elif isinstance(row["date"], pd.Timestamp):
        return row["date"].to_pydatetime()
    else:
        logger.warning("Failed to extract date".format(row['date']), "which is of type".format(type(row['date'])) + "Returning None ..")
        return None
    
def date_filter_pandas_wrapper(date, datetime_threshold):
    try:
        return date >= datetime_threshold
    except TypeError:
        return datetime(date.year, date.month, date.day, tzinfo=timezone.utc) >= datetime_threshold    

def get_window_filter_idx(original_list, window_size=3, criteria=lambda x: True):
    new_list = []
    max_idx = len(original_list) - 1

    for idx, val in enumerate(original_list):
        if criteria(val):
            new_list.append(idx)

            i = window_size
            while i > 0:
                new_list.append(max(idx - i, 0))
                new_list.append(min(idx + i, max_idx))
                i -= 1

    new_list = list(set(new_list))

    return new_list


def dict_to_tupes_wrapper(dict_input, avg_top_n_scores=3):
    tupe_list = []

    for q in dict_input:
        doc_idx, scores = zip(*dict_input[q])
        top_avg = np.mean(scores[:avg_top_n_scores])
        tupe_list.append((q, top_avg))

    return tupe_list

#functionalize entity count secondary eval rule
import string 
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
newStopWords = ['.','?','%','&','Google','Wells Fargo','Donald Trump','Charles Schwab','Morgan Stanley','Credit Suisse','Reuters','Bank of America','Guggenheim','Deutsch Bank','Goldman Sachs','Facebook','Fifth Third Bank','New York','Washington','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','January','February','March','April','May','June','July','August','September','October','November','December']
remove_list = ['Bank','Securities','Inc','Life','.com']
stopwords.extend(newStopWords+remove_list)

def entity_count_eval(row,Stopwords=stopwords):
    ent_name_list = [word for word in re.sub('[^0-9a-zA-Z]+', ' ',row['entity'].lower()).split() if word.lower() not in [x.lower() for x in Stopwords]]
    source =re.sub('[^0-9a-zA-Z]+', '',row['source'].lower().replace(" ",""))
    title =  re.sub('[^0-9a-zA-Z]+', '',row['title'].lower().replace(" ",""))
  
    val_assign = 0
    if ((row['entity_count'] == 1) and (any(val in title for val in ent_name_list)) 
    and not any(val in source for val in ent_name_list)):
        #print('1 Entity Mention',source,' ',title)
        val_assign = 1
    if((row['entity_count'] > 1) and not any(val in source for val in ent_name_list)):
        #print('>1 Entity Mention',source,' ',title)
        val_assign = 2
    return val_assign

def dict_to_tupes_wrapper(dict_input, avg_top_n_scores=3):
    tupe_list = []

    for q in dict_input:
        doc_idx, scores = zip(*dict_input[q])
        top_avg = np.mean(scores[:avg_top_n_scores])
        tupe_list.append((q, top_avg))

    return tupe_list

def normalize_pandas_series(series, _max=None, _min=None):
    if _max is None:
        if series.max() == 0:
            _max = 1
        else:
            _max = series.max()

    if _min is None:
        _min = series.min()

    return (series - _min) / (_max - _min)

def aggregate_into_single_list(row, col_name_list):
    final_list = []
    for col in col_name_list:
        if type(row[col]) == list:
            for item in row[col]:
                final_list.append(item)
        else:
            final_list.append(row[col])
    return final_list



def get_highest_fine_bin(fine_tuple_list, is_org=1):
    _list = []

    for fine_tuple in fine_tuple_list:

        orgs_from_title, total_fine, generate_flag, _, pct_equity_list = fine_tuple

        if generate_flag:
            for pct in pct_equity_list:
                if pct > 0.1:
                    _list.append(("equity", 7))
                elif ((is_org == 1) & (total_fine > 500000)) | ((is_org == 0) & (total_fine > 100000)):
                    _list.append(("fine threshold", 3))
                else:
                    _list.append(("other", 1))
        else:
            _list.append(("other", 1))

    highest_bin = sorted(_list, key=lambda x: x[1], reverse=True)[0]

    return highest_bin

def reg_site_classifier(url):
    return any(k in url for k in regulatory_url_list)

def make_classification_tuples(wv, row, clf_metadata_dict=ARTICLE_CLASSES_DICT):
    CLF_NAME_NONE_FOUND = "other"

    CLF_NAME_MINOR_FINE = "monetary_fine_minor"
    FLAG_MINOR_FINE = "equity"

    CLF_NAME_MAJOR_FINE = "monetary_fine_major"
    FLAG_MAJOR_FINE = "fine threshold"

    clf_tuple_list = []

    for clf_name, clf_metadata in clf_metadata_dict.items():
        if clf_metadata["stems"] != []:
            max_term_similarity = max(
                [
                    max(np.concatenate(wv.similarity(clf_metadata["stems"], token_list)))
                    for token_list in row["sentence_tokenized_word_tokenized"]
                    if token_list != []
                ]
            )
        else:
            max_term_similarity = 0

        if (max_term_similarity > clf_metadata["threshold"]) or \
        any(s in  ' '.join(row["sentence_tokenized"]) for s in clf_metadata_dict[clf_name]["static_phrases"]):
            clf_tuple_list.append((clf_metadata["code"], clf_name))

    # this needs to be refactored
    # should extract fine percent of equity into a separate column and then
    # check whether fine percent of equity exceeds user-defined threshold
    # which can be done in the 'make_classification_tuples' function (this function)
    clf_names_from_stem_similarity = [item[1] for item in clf_tuple_list]

    if row["fine_bins"][0] == FLAG_MAJOR_FINE and CLF_NAME_MAJOR_FINE not in clf_names_from_stem_similarity:
        clf_metadata = clf_metadata_dict[CLF_NAME_MAJOR_FINE]
        clf_tuple_list.append((clf_metadata["code"], CLF_NAME_MAJOR_FINE))
    elif row["fine_bins"][0] == FLAG_MINOR_FINE and CLF_NAME_MINOR_FINE not in clf_names_from_stem_similarity:
        clf_metadata = clf_metadata_dict[CLF_NAME_MINOR_FINE]
        clf_tuple_list.append((clf_metadata["code"], CLF_NAME_MINOR_FINE))

    if clf_tuple_list == []:
        clf_metadata = clf_metadata_dict[CLF_NAME_NONE_FOUND]
        clf_tuple_list.append((clf_metadata["code"], CLF_NAME_NONE_FOUND))

    return clf_tuple_list

#one liner to fix issue with lazyconfig load 
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""


### Analysis Starts Here

##### Extract Source Data from Mongo

In [37]:
#list all collections in each database (2 ways)
#client = MongoClient("172.24.0.4:27017")
client = MongoClient("172.30.0.3:27017")

db_collections = dict((db, [collection for collection in client[db].collection_names()])
                      for db in client.database_names())
print(db_collections)

#get data into dataframe 
coll = client['negative_news']['scrape_feed']
data = list(coll.find({}))
data = pd.DataFrame(data)
print(data.shape)
data.dropna(subset=["content"], inplace=True)

{'local': ['startup_log'], 'negative_news': ['rss_feed', 'broker_check_owners', 'broker_check_meta', 'broker_check_disclosures', 'scrape_feed', 'processed_results'], 'admin': ['system.version'], 'config': ['system.sessions']}
(13378, 11)


In [38]:
# convert string dates/fill empty dates
data["date"] = data.apply(lambda x: date_extract_pandas_wrapper(x), axis=1)
NULL_DATE_DATA = data[data['date'].isnull()]
print('NULL DATES:',NULL_DATE_DATA.shape)
data.dropna(subset=["date"], inplace=True)
#save output to pickle for later use
#NULL_DATE_DATA.to_pickle("{}".format(OUTPUT_PATH)+"NULL_DATE_DATA.pickle")
#data.to_pickle("{}".format(OUTPUT_PATH)+"SourceFeedDump.pickle")

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.5/logging/__init__.py", line 980, in emit
    msg = self.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 830, in format
    return fmt.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 567, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.5/logging/__init__.py", line 330, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.5/logging/__init__.py", line 980, in emit
    msg = self.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 830, in format
    return fmt.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 567, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.5/logging/__init__.py", line 330, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.5/logging/__init__.py", line 980, in emit
    msg = self.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 830, in format
    return fmt.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 567, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.5/logging/__init__.py", line 330, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.5/logging/__init__.py", line 980, in emit
    msg = self.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 830, in format
    return fmt.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 567, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.5/logging/__init__.py", line 330, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python

  File "/usr/local/lib/python3.5/dist-packages/pandas/core/apply.py", line 151, in get_result
    return self.apply_standard()
  File "/usr/local/lib/python3.5/dist-packages/pandas/core/apply.py", line 251, in apply_standard
    labels=labels)
  File "<ipython-input-38-9674bb76148c>", line 2, in <lambda>
    data["date"] = data.apply(lambda x: date_extract_pandas_wrapper(x), axis=1)
  File "<ipython-input-15-8d144f9777a2>", line 17, in date_extract_pandas_wrapper
    logger.warning("Failed to extract date".format(row['date']), "which is of type".format(type(row['date'])) + "Returning None ..")
Message: 'Failed to extract date'
Arguments: ('which is of typeReturning None ..',)
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.5/logging/__init__.py", line 980, in emit
    msg = self.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 830, in format
    return fmt.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 567, in 

...

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.5/logging/__init__.py", line 980, in emit
    msg = self.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 830, in format
    return fmt.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 567, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.5/logging/__init__.py", line 330, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python

  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelbase.py", line 357, in process_one
    yield gen.maybe_future(dispatch(*args))
  File "/usr/local/lib/python3.5/dist-packages/tornado/gen.py", line 307, in wrapper
    yielded = next(result)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelbase.py", line 267, in dispatch_shell
    yield gen.maybe_future(handler(stream, idents, msg))
  File "/usr/local/lib/python3.5/dist-packages/tornado/gen.py", line 307, in wrapper
    yielded = next(result)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelbase.py", line 534, in execute_request
    user_expressions, allow_stdin,
  File "/usr/local/lib/python3.5/dist-packages/tornado/gen.py", line 307, in wrapper
    yielded = next(result)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/ipkernel.py", line 294, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.5/logging/__init__.py", line 980, in emit
    msg = self.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 830, in format
    return fmt.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 567, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.5/logging/__init__.py", line 330, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python

NULL DATES: (454, 11)


Traceback (most recent call last):
  File "/usr/lib/python3.5/logging/__init__.py", line 980, in emit
    msg = self.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 830, in format
    return fmt.format(record)
  File "/usr/lib/python3.5/logging/__init__.py", line 567, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.5/logging/__init__.py", line 330, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipyk

In [39]:
print('remove dates before date threshold:{},"-"{},"-"{}'.format(MIN_YEAR,MIN_MONTH,MIN_DAY))
print('shape BEFORE removing old dates:{}'.format(data.shape))

date_threshold = datetime(MIN_YEAR, MIN_MONTH, MIN_DAY)
data = data[data.apply(lambda x: date_filter_pandas_wrapper(x["date"], date_threshold), axis=1)]
print('shape AFTER removing old dates:{}'.format(data.shape))

#[NULL_DATE_DATA[col].value_counts() for col in ['url','title','entity']]
#data[['entity','date','title','url','content']][(data['date']>datethresh) & (data['title'].str.contains('Wedbush'))]

remove dates before date threshold:2000,"-"1,"-"1
shape BEFORE removing old dates:(12924, 11)
shape AFTER removing old dates:(12605, 11)


In [40]:
#tokenize content (sentence and word)
data["sentence_tokenized"] = data.apply(lambda x: sent_tokenizer(x["content"]), axis=1)
data["sentence_tokenized_word_tokenized"] = data.apply(
    lambda x: list(map(lambda y: word_tokenizer(y), x["sentence_tokenized"])), axis=1)

In [41]:
# for each article, determine how many of sentence-level fuzzy scores exceed the threshold
from fuzzywuzzy import fuzz
import re
data["name_fuzzy_score_list"] = data.apply(
    lambda x: list(map(lambda sent: max([fuzz.partial_ratio(sent, name) for name in [x['entity']]]), x["sentence_tokenized"])), axis=1
)

#check content for entity count
data["entity_count"] = data.apply(
    lambda x: sum(list(map(lambda score: score > ENTITY_FUZZ_SCORE_THRESHOLD, x["name_fuzzy_score_list"]))), axis=1
)

#sum total instances found in content and in title 
#apply 2nd entitycount function
data['keep']= data.apply(lambda x: entity_count_eval(x),axis=1)
print('done')

done


In [42]:
#remove articles with no mention of entity 
df = data[data["keep"] > 0].copy()
df_drop = data[data["keep"]==0].copy()
print('Total Rows removed as a result of no entity mentions: {}'.format(df_drop.shape[0]))

Total Rows removed as a result of no entity mentions: 564


In [43]:
# generate sentence indices to keep based on window filtering method
df["window_filter_idx_list"] = df.apply(
    lambda x: get_window_filter_idx(x["name_fuzzy_score_list"], criteria=lambda score: score > ENTITY_FUZZ_SCORE_THRESHOLD), axis=1
)

# filter sentence tokens based on the results of our window filter indices
df["sentence_tokenized"] = df.apply(lambda x: [x["sentence_tokenized"][idx] for idx in x["window_filter_idx_list"]], axis=1)

# filter word-tokenized, sentence tokens based on the results of our window filter indices
df["sentence_tokenized_word_tokenized"] = df.apply(
    lambda x: [x["sentence_tokenized_word_tokenized"][idx] for idx in x["window_filter_idx_list"]], axis=1
)

# remove stop words and punctuation from word-tokenized sentence tokens
df["sentence_tokenized_word_tokenized"] = df.apply(
        lambda row: [
            [word for word in sent if word.lower() not in stopwords and word not in punctuation]
            for sent in row["sentence_tokenized_word_tokenized"]
        ],
        axis=1,
    )

print('done')

done


In [44]:
#SUBET REMAINING ANALYSIS AT THE ENTITY LEVEL TO ENABLE DESM CALCS 
df.entity.value_counts()
#entity_name='Wedbush Securities'
#is_org = 1
#df_sub = df[df['entity']==entity_name]

df_sub = df
df_sub["sentence_tokenized_english"] = df_sub["sentence_tokenized"]

In [8]:
#calc desm scores
import sys
import pymagnitude
sys.path.insert(0, "//home//nlpsomnath//NegNews//zackc//Misc Notebooks//NN_VM//lib//python3.7//site-packages//negative_news//negative_news//util//")
from desm import desm_pandas_wrapper

wv=pymagnitude.Magnitude('//home//nlpsomnath//NegNews//rebase//NN_Kafka//sourcefiles//GoogleNews-vectors-negative300.magnitude')

search_terms = ['fined','penalty','sanction','violation','revoke','suspension','restitution','escalation', 'infringement', 'collude','deficiencies']
search_terms_desm = search_terms 

df_sub["max_desm_score"], df_sub["desm_sentences"], df_sub["desm_term"], df_sub["desm_score_dict"] = zip(
        *df_sub.apply(lambda x: desm_pandas_wrapper(wv, search_terms, x), axis=1)
    )

df_sub.sort_values("max_desm_score", ascending=False, inplace=True)

In [47]:
import numpy as np
df_sub["term_tupes"] = df_sub.apply(lambda x: dict_to_tupes_wrapper(x["desm_score_dict"]), axis=1)

In [48]:
# make SentimentIntensityAnalyzer object for the purpose
# of generating VADER sentiment scores
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# this constant represents the top N most negative sentences
# whose sentiment scores will be averaged to calculate
# the overall article sentiment
TAKE_N_MOST_NEGATIVE_SENTENCES = 3

# calculate a sentiment score for each sentence in each document
sentence_sentiment_list = []
doc_sentiment_list = []
for doc_idx, doc in enumerate(df_sub["sentence_tokenized_english"]):
    sentence_sentiment_list.append([])
    for i_idx, item in enumerate(doc):
        try:
            sentence_sentiment_list[doc_idx].append(analyzer.polarity_scores(item)["compound"])
        except Exception:
            logger.debug("Sentiment calculation failed at doc index {}. Sentence is: {}".format(doc_idx, item))
            sentence_sentiment_list[doc_idx].append(0)
    doc_sentiment_list.append(np.mean(sorted(sentence_sentiment_list[doc_idx])[:TAKE_N_MOST_NEGATIVE_SENTENCES]))

# assign sentiment
# TODO: we may be able to remove this sentence_sentiment column (not used anywhere)
df_sub["sentence_sentiment"] = sentence_sentiment_list
df_sub["doc_sentiment"] = doc_sentiment_list

In [49]:
import spacy
import time
from fines import get_data_from_article_title, transform_article_title, load_entity_dict, generate_report_string, max_marketshare
nlp=spacy.load("en_core_web_sm", disable=["tagger", "parser"])
# adjust this constant to the name of the column containing the top sentences
TOP_SENTENCES_COL_NAME = "desm_sentences"

# NER tag desired columns
logger.debug("Beginning NER tagging via spaCy ..")
_tock = time.perf_counter()
df_sub["tagged_top_sentences"] = df_sub.apply(lambda x: list(map(lambda sent: nlp(sent), x[TOP_SENTENCES_COL_NAME])), axis=1)
df_sub["tagged_title"] = df_sub.apply(lambda x: nlp(transform_article_title(x["title"]), " ".join(x[TOP_SENTENCES_COL_NAME])), axis=1)
df_sub["tagged_summary"] = df_sub.apply(lambda x: nlp(x["summary"]), axis=1)
_tick = time.perf_counter()
logger.debug("Finished NER tagging via spaCy. Took {}".format(round(_tick - _tock, 2)))

In [55]:
#entity_name='Wedbush Securities'
is_org = 1
df_sub2 = df_sub#df_sub[df_sub['entity']==entity_name]
df_sub2.shape
#df_sub.to_pickle("{}".format(OUTPUT_PATH)+"DF_With_DESM_Scores.pickle")

In [56]:
# generate fine tuples (e.g., (['ent1', 'ent2'], 1000000000.0, True) )
df_sub2["fine_tuples_top_sentences"] = df_sub2.apply(
    lambda x: [
        get_data_from_article_title(entity_dict, tagged_text=sent, entity_name=x["entity"], use_stemmer=False)
        for sent in x["tagged_top_sentences"]
    ],
    axis=1,
)

df_sub2["fine_tuples_title"] = df_sub2.apply(
    lambda x: get_data_from_article_title(entity_dict, tagged_text=x["tagged_title"], entity_name=x["entity"], use_stemmer=False),
    axis=1,
)

df_sub2["fine_tuples_summary"] = df_sub2.apply(
    lambda x: get_data_from_article_title(
        entity_dict, tagged_text=x["tagged_summary"], entity_name=x["entity"], use_stemmer=False
    ),
    axis=1,
)

# aggregate fine tuples into one list for each row
df_sub2["fine_tuples_aggregated"] = df_sub2.apply(
    lambda row: aggregate_into_single_list(row, ["fine_tuples_top_sentences", "fine_tuples_title", "fine_tuples_summary"]), axis=1
)

# sum total fine amount from fine tuples
FINE_AMOUNT_IDX = 1
FINE_GENERATE_FLAG_IDX = 2
df_sub2["total_entity_fine_amt"] = df_sub2.apply(
    lambda x: sum(set([t[FINE_AMOUNT_IDX] if t[FINE_GENERATE_FLAG_IDX] else 0 for t in x["fine_tuples_aggregated"]])), axis=1
)

# generate fine report strings
df_sub2["fine_report_strings"] = df_sub2.apply(
    lambda row: list(
        set([fr for ft in row["fine_tuples_aggregated"] for fr in generate_report_string(fine_tuple=ft, entity_dict=entity_dict)])
    ),
    axis=1,
)

df_sub2["fine_bins"] = df_sub2.apply(lambda row: get_highest_fine_bin(row["fine_tuples_aggregated"], is_org), axis=1)

# normalize fine amount and document age relative to itself
df_sub2["fine_amt_norm"] = df_sub2.apply(lambda row: max_marketshare(row["fine_tuples_aggregated"], mktshare_tuploc=4), axis=1)
df_sub2["fine_amt_norm"] = normalize_pandas_series(df_sub2["fine_amt_norm"], _max=0.1, _min=0)

In [57]:
# for each article, generate fuzzy scores for each sentence for location and employment
location = None
employment = None
df_sub2["location_fuzzy_score_list"] = df_sub2.apply(
    lambda row: [fuzz.partial_token_sort_ratio(location, sent) for sent in row["sentence_tokenized"]], axis=1
)
df_sub2["employment_fuzzy_score_list"] = df_sub2.apply(
    lambda row: [fuzz.partial_token_sort_ratio(employment, sent) for sent in row["sentence_tokenized"]], axis=1
)

# for each article; take the average of the name, location, and employment sentence scores to generate
# the final scores for each category
df_sub2["name_fuzzy_score_final"] = df_sub2.apply(lambda row: np.mean(row["name_fuzzy_score_list"]), axis=1)
df_sub2["location_fuzzy_score_final"] = df_sub2.apply(lambda row: np.mean(row["location_fuzzy_score_list"]), axis=1)
df_sub2["employment_fuzzy_score_final"] = df_sub2.apply(lambda row: np.mean(row["employment_fuzzy_score_list"]), axis=1)


In [11]:
#df_sub2 = pd.read_pickle("{}".format(OUTPUT_PATH)+"DF_With_DESM_Scores.pickle")
#classified_df =pd.read_pickle("{}".format(OUTPUT_PATH)+" DF_With_DESM_Scores_AND_CLASSIFICATIONS.pickle")
df_sub3 = df_sub2
#df_sub3["regulatory_flag"] = df_sub3.apply(lambda x: reg_site_classifier(x["url"]), axis=1)
#df_sub3["classification_tuple_lists"] = df_sub3.apply(lambda row: make_classification_tuples(wv, row), axis=1)
#df_sub3.to_pickle("{}".format(OUTPUT_PATH)+"DF_With_DESM_Scores.pickle")


In [2]:
#df_sub3.columns#drop(['classification_tuple_lists','regulatory_flag'],axis=1,inplace=True)

In [10]:
import numpy as np
df_sub3["claassification_tuple_lists"] = df_sub3.apply(lambda row: make_classification_tuples(wv, row), axis=1)

### END SCRIPT

In [ ]:
from datetime import datetime, timezone, timedelta
import time
entity='MUFG'
datethresh = datetime(2018, 12, 1)

df_sub3 = df_sub2[df_sub2['entity']==entity]
df_sub3.shape

In [69]:

def make_classification_tuples(wv, row, clf_metadata_dict=ARTICLE_CLASSES_DICT):
    CLF_NAME_NONE_FOUND = "other"

    CLF_NAME_MINOR_FINE = "monetary_fine_minor"
    FLAG_MINOR_FINE = "equity"

    CLF_NAME_MAJOR_FINE = "monetary_fine_major"
    FLAG_MAJOR_FINE = "fine threshold"

    clf_tuple_list = []

    for clf_name, clf_metadata in clf_metadata_dict.items():
        if clf_metadata["stems"] != []:
            max_term_similarity = max(
                [
                    max(np.concatenate(wv.similarity(clf_metadata["stems"], token_list)))
                    for token_list in row["sentence_tokenized_word_tokenized"]
                    if token_list != []
                ]
            )
        else:
            max_term_similarity = 0

        if (max_term_similarity > clf_metadata["threshold"]) or \
        any(s in  ' '.join(row["sentence_tokenized"]) for s in clf_metadata_dict[clf_name]["static_phrases"]):
            clf_tuple_list.append((clf_metadata["code"], clf_name))

    # this needs to be refactored
    # should extract fine percent of equity into a separate column and then
    # check whether fine percent of equity exceeds user-defined threshold
    # which can be done in the 'make_classification_tuples' function (this function)
    clf_names_from_stem_similarity = [item[1] for item in clf_tuple_list]

    if row["fine_bins"][0] == FLAG_MAJOR_FINE and \
    CLF_NAME_MAJOR_FINE not in clf_names_from_stem_similarity:
        clf_metadata = clf_metadata_dict[CLF_NAME_MAJOR_FINE]
        clf_tuple_list.append((clf_metadata["code"], CLF_NAME_MAJOR_FINE))

        # filter out any minor fine tuples
        clf_tuple_list = [item for item in clf_tuple_list if item[1] != CLF_NAME_MINOR_FINE]

    elif row["fine_bins"][0] == FLAG_MINOR_FINE and \
    CLF_NAME_MINOR_FINE not in clf_names_from_stem_similarity:
        clf_metadata = clf_metadata_dict[CLF_NAME_MINOR_FINE]
        clf_tuple_list.append((clf_metadata["code"], CLF_NAME_MINOR_FINE))

    if not len(clf_tuple_list):
        max_classification_score = 0
        clf_metadata = clf_metadata_dict[CLF_NAME_NONE_FOUND]
        clf_tuple_list.append((clf_metadata["code"], CLF_NAME_NONE_FOUND))
    else:
        max_classification_score = max([item[0] for item in clf_tuple_list])

    return clf_tuple_list, max_classification_score


In [ ]:
df_sub3=df_sub3[(df_sub3['date']>datethresh) | (df_sub3['date'].isnull())]
df_sub3['date'].unique()

In [ ]:
df_sub3["regulatory_flag"] = df_sub3.apply(lambda x: reg_site_classifier(x["url"]), axis=1)
df_sub3["classification_tuple_lists"] = df_sub3.apply(lambda row: make_classification_tuples(wv, row), axis=1)
#df_sub3.to_pickle("{}".format(OUTPUT_PATH)+"FinalDF_WithClassifications.pickle")

In [79]:
def make_classification_tuples(wv, row, clf_metadata_dict=ARTICLE_CLASSES_DICT):
    CLF_NAME_NONE_FOUND = "other"

    CLF_NAME_MINOR_FINE = "monetary_fine_minor"
    FLAG_MINOR_FINE = "equity"

    CLF_NAME_MAJOR_FINE = "monetary_fine_major"
    FLAG_MAJOR_FINE = "fine threshold"

    clf_tuple_list = []

    for clf_name, clf_metadata in clf_metadata_dict.items():
        if clf_metadata["stems"] != []:
            max_term_similarity = max(
                [
                    max(np.concatenate(wv.similarity(clf_metadata["stems"], token_list)))
                    for token_list in row["sentence_tokenized_word_tokenized"]
                    if token_list != []
                ]
            )
        else:
            max_term_similarity = 0

        if (max_term_similarity > clf_metadata["threshold"]) or \
        any(s in  ' '.join(row["sentence_tokenized"]) for s in clf_metadata_dict[clf_name]["static_phrases"]):
            clf_tuple_list.append((clf_metadata["code"], clf_name))

    # this needs to be refactored
    # should extract fine percent of equity into a separate column and then
    # check whether fine percent of equity exceeds user-defined threshold
    # which can be done in the 'make_classification_tuples' function (this function)
    clf_names_from_stem_similarity = [item[1] for item in clf_tuple_list]

    if row["fine_bins"][0] == FLAG_MAJOR_FINE and CLF_NAME_MAJOR_FINE not in clf_names_from_stem_similarity:
        clf_metadata = clf_metadata_dict[CLF_NAME_MAJOR_FINE]
        clf_tuple_list.append((clf_metadata["code"], CLF_NAME_MAJOR_FINE))
    elif row["fine_bins"][0] == FLAG_MINOR_FINE and CLF_NAME_MINOR_FINE not in clf_names_from_stem_similarity:
        clf_metadata = clf_metadata_dict[CLF_NAME_MINOR_FINE]
        clf_tuple_list.append((clf_metadata["code"], CLF_NAME_MINOR_FINE))

    if clf_tuple_list == []:
        clf_metadata = clf_metadata_dict[CLF_NAME_NONE_FOUND]
        clf_tuple_list.append((clf_metadata["code"], CLF_NAME_NONE_FOUND))

    return clf_tuple_list

In [ ]:
test = df_sub3[['date','url','sentence_tokenized',"classification_tuple_lists"]][df_sub3['date'].isnull()==False]#[df_sub3['url']=='https://stockbrokerfraud.com/wedbush-chairman-investigated/']
#[df_sub3['content'].str.contains('50 states')]#
#test.iloc[1,:]
#list(df_sub3)
#test['barred'] = test.apply(lambda x: ' '.join(x['sentence_tokenized']),axis=1)
#test.loc[test.index[0], 'sentence_tokenized']
#test['barred'] 


In [ ]:
a =test['sentence_tokenized']
values = ' '.join(str(v) for v in test['sentence_tokenized'])
#' '.join([x for x in test['sentence_tokenized'].values])
#["violation of federal securities laws" in ' '.join(test['sentence_tokenized'])]

#filepath = "{}".format(OUTPUT_PATH)+"FinalDF_WithClassifications.pickle"
#data=pickle.load( open( filepath, "rb" ) )
#data = pd.DataFrame(data)

In [ ]:
tuplist= list(df_sub3['classification_tuple_lists'])
all_tup_vals = [item for sublist in tuplist for item in sublist]
RiskCatDF = pd.DataFrame(all_tup_vals, columns=['RiskScore', 'RiskCat'])
RiskCatDF.pivot(index=None,columns='RiskCat',values='RiskScore').fillna(0)
RiskCatDF['RiskCat'].value_counts()

In [ ]:
list(data)
'desm_score_dict','sentence_sentiment'

for col in list(data):
    print(col,":",data[col].head(1))

In [ ]:
RiskCatDF

In [ ]:
output = []
def reemovNestings(l):
    for i in l: 
        if type(i) == list: 
            reemovNestings(i) 
        else: 
            output.append(i) 


#### ENTITY COUNT RULE EVALUTION 

In [ ]:
#lets see the sources tied to entity_counts ==1
data[data['entity_count']==1]['source'].value_counts()
df = pd.melt(data[data['entity_count']==1], id_vars=['entity','source','title','date'], var_name='Name')

#evaluate entity counts across sources where entity name only mentioned 1 time 
#RULE: if entity name in title and entity_count=1 and entity not in source 
singleEntMentions = df[df['Name']=='entity_count']

#RULE LOGIC TO IMPLEMENT
singleEntMentions['NameinTitle'] = singleEntMentions.apply(
    lambda x: (x['entity'].lower() in x['title'].lower()) 
    and (x['entity'].lower() not in x['source'].lower()),axis=1)

#view results 
singleEntMentions[singleEntMentions['NameinTitle']==True]

#evaluate final entity count function output
a =data.apply(lambda x: entity_count_eval(x,Stopwords =stopwords),axis=1)
a.apply(pd.Series).stack().value_counts()

#CHECK SOURCES AND LINKS THAT ARE FILTERED OUT DUE TO ENTITY COUNT
#data[data['keep']==0]['source'].value_counts()#[['entity','title','source']]
#data[(data['keep']==0) & (data['source']=='The New York Times')][['entity','title']]
#data[(data['keep']==0)][['entity','title','source']]

In [ ]:

# data = pd.read_pickle("/home/nlpsomnath/NegNews/zackc/Misc Notebooks/NN_VM/lib/python3.7/site-packages/negative_news_dev/test/output/02-19-19-01-17-23/Wells Fargo_02-19-19-01-17-23/postprocess_df.pk")
# data_args = pd.read_pickle("/home/nlpsomnath/NegNews/zackc/Misc Notebooks/NN_VM/lib/python3.7/site-packages/negative_news/test/output/02-19-19-01-17-23/Wells Fargo_02-19-19-01-17-23/postprocess_args.pk")
# data_args = pd.read_pickle("/home/nlpsomnath/NegNews/nn_env/lib/python3.7/site-packages/negative_news/negative_news/output/02-20-19-13-17-29/Wells Fargo_02-20-19-13-17-29/postprocess_args.pk")
# data = pd.read_pickle("/home/nlpsomnath/NegNews/zackc/Misc Notebooks/NN_VM/lib/python3.7/site-packages/negative_news_dev/negative_news/output/02-21-19-12-14-40/Wells Fargo_02-21-19-12-14-40/postprocesspt2_data.pk")

# data_path="/home/nlpsomnath/NegNews/zackc/Misc Notebooks/NN_VM/lib/python3.7/site-packages/negative_news_dev/test/output/02-21-19-13-31-01/Bank of America_02-21-19-13-31-01/"
# file1 = "postprocess_df.pk"
# file2 = "postprocesspt2_data.pk"
# post_data = pd.read_pickle(data_path+file1)
# post2_data = pd.read_pickle(data_path+file2)


# import pickle 
# filepath = "/home/nlpsomnath/NegNews/zackc/postprocess_datadic.pk"
# data=pickle.load( open( filepath, "rb" ) )
# subset=[item for item in data['scraped_data'] if pd.to_datetime(item['date'], errors='coerce').replace(tzinfo=utc)< datetime(2015, 12, 1).replace(tzinfo=utc)]
# filterd = [item for item in data if item['scraped_data'][0]['jobname'] == 'Wells Fargo_02-27-19-19-13-35'] 